In [1]:
import json
import re
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

# x.

In [2]:
def get_ip_info():
    """output ip_list -> List 
    
       gets ip address from emails, 
       returns a list containing risk score and risk level of the ip address"""
    
    ip_list = []
    
    for i in tqdm(range(1, 4030)):
        ip_dict = {'index': i, 'ip': 'unknown'}
        
        with open(f'../../data/separated_by_email/{i}.json') as json_file:
            x = json.load(json_file)
        
            if 'MboxParser-x-originating-ip' in x.keys():
                ip_original = x['MboxParser-x-originating-ip']
            elif 'MboxParser-x-sender-ip' in x.keys():
                ip_original = x['MboxParser-x-sender-ip']
            # if no ip address found, directly add features {ip: 'unknown'}
            else: 
                ip_list.append(ip_dict)
                continue
        
        ip_address = re.search(r'\d+\.\d+\.\d+\.\d+', ip_original).group()
        
        # search ip address on scamalytics.com for risk level
        url = f'https://scamalytics.com/ip/{ip_address}'
        content = requests.get(url)
        soup = BeautifulSoup(content.content, features='html.parser')
        content.close()

        risk = soup.find('pre').text
        risk_dict = json.loads(risk)
        ip_dict = {'index': i, 'ip_score': risk_dict['score'], 'ip_risk': risk_dict['risk']}
        ip_list.append(ip_dict)
        
    return ip_list

In [3]:
# this takes 5-10 mins 
ip_list = get_ip_info()

  0%|          | 0/4029 [00:00<?, ?it/s]

In [4]:
ip_list

[{'index': 1, 'ip': 'unknown'},
 {'index': 2, 'ip': 'unknown'},
 {'index': 3, 'ip': 'unknown'},
 {'index': 4, 'ip': 'unknown'},
 {'index': 5, 'ip': 'unknown'},
 {'index': 6, 'ip': 'unknown'},
 {'index': 7, 'ip_score': '8', 'ip_risk': 'low'},
 {'index': 8, 'ip': 'unknown'},
 {'index': 9, 'ip': 'unknown'},
 {'index': 10, 'ip_score': '8', 'ip_risk': 'low'},
 {'index': 11, 'ip': 'unknown'},
 {'index': 12, 'ip': 'unknown'},
 {'index': 13, 'ip': 'unknown'},
 {'index': 14, 'ip': 'unknown'},
 {'index': 15, 'ip': 'unknown'},
 {'index': 16, 'ip': 'unknown'},
 {'index': 17, 'ip': 'unknown'},
 {'index': 18, 'ip': 'unknown'},
 {'index': 19, 'ip': 'unknown'},
 {'index': 20, 'ip': 'unknown'},
 {'index': 21, 'ip': 'unknown'},
 {'index': 22, 'ip': 'unknown'},
 {'index': 23, 'ip': 'unknown'},
 {'index': 24, 'ip': 'unknown'},
 {'index': 25, 'ip': 'unknown'},
 {'index': 26, 'ip': 'unknown'},
 {'index': 27, 'ip': 'unknown'},
 {'index': 28, 'ip': 'unknown'},
 {'index': 29, 'ip': 'unknown'},
 {'index': 30, '

In [5]:
def save_to_local(data, filepath):
    """input data -> List/Dict 
             filepath -> Str
             
       save features of each email in separated json files to the input filepath"""
    for i in range(0, 4030):
        with open(f'{filepath}/5b10_{i}.json', 'w') as json_file:
            json.dump(data, json_file)

In [6]:
def upload_to_firebase(data):
    """input data -> List
    
       upload input data to firebase"""
    for i in range(0, 4029):
        requests.patch(f'https://copydsci550.firebaseio.com/{i+1}.json', json=data[i])

save features as json files to local path

In [7]:
# path = '?' # enter the path you want to save the output
# save_to_local(ip_list, path)

upload features to firebase<br>
(our database on firebase is currently read-only)

In [8]:
# upload_to_firebase(ip_list)